In [1]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
import utilities as ut
from Thymio import Thymio
import os
import time
import serial
from AstolfiController import Astolfi
from SimpleController import SimpleController
from master import Vision

In [2]:

from Thymio import Thymio
th = Thymio.serial(port='COM9', refreshing_rate=0.1)

In [ ]:
th.set_var("motor.left.target", 100)
th.set_var("motor.right.target", 100)

In [ ]:
th.set_var("motor.left.target", 100)
th.set_var("motor.right.target", 2**16- 100)
#ccw

In [ ]:
th.set_var("motor.left.target", 2**16-100)
th.set_var("motor.right.target", 100)
#cw

In [5]:
th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)

In [ ]:
vis = Vision(video_source=1)
flag = False
while flag==False:

    temp = vis.prepareWaypoints()

    if temp is not None:
        angle = temp[0]
        trajectory =temp[1]
        flag = True

In [6]:
trajectory = [(0.0,0.0),(20.0,0.0),(40.0,20.0),(60.0,30.0)]
# trajectory = [(0.0,0.0),(5.0,0.0),(10.0,5.0),(10.0,0.0),(0.0,0.0)]
# trajectory = [(0.0,0.0),(3.0,0.0),(3.0,3.0),(0.0,3.0),(0.0,0.0)]
# trajectory = [(0.0,0.0), (1.0,-1.0), (2.0,-1.5), (2.5, 1.5), (1.0,3.0),(0.0,0.0)]
# trajectory = [(0,0), (5,1), (10,1) ]

# curr=np.array((trajectory[0][0]/10,trajectory[0][1]/10,np.deg2rad(angle)))

curr=np.array((0.0, 0.0, m.pi/2))
angle=0.0

astol = Astolfi()
astol.set_path(trajectory,angle)
# curr=astol.path[0]
# curr[2]=astol.normalize_ang(np.deg2rad(angle))

astol.set_curr(curr)
simp = SimpleController(th)

simp.set_curr(curr)
simp.set_global(trajectory,angle)
pba=[]
pos=[]
phi_dot=[]
i=0
simp.set_thymio(th)
simp.correct_heading(th)
# print(astol.curr)
print((astol.path))

while astol.on_goal==False and simp.on_goal==False:  

        astol.check_nodes(verbose=True)
        rho,alpha,beta = astol.polar_rep()
        if astol.rho < 0.5:
            pos.insert(i,[simp.curr[0],simp.curr[1],simp.curr[2]])
            simp.follow_line()
            simp.check_node()
            simp.correct_heading(th)
            astol.set_curr(simp.get_curr())
            astol.set_goal(simp.get_goal())
        
        elif astol.rho >= 0.5:
            pos.insert(i,[astol.curr[0],astol.curr[1],astol.curr[2]])      
            
            phiL,phiR = astol.compute_phi_dot(fkine=True)
            phi_dot.append([phiL, -phiR])
            astol.run_on_thymio(th)
            simp.set_curr(astol.get_curr())  
            simp.set_goal(astol.get_goal())
            pba.append([rho,alpha,beta])      

        
        print(astol.curr)    
        time.sleep(astol.Ts)
        i+=1
th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)
pos.insert(i,[astol.curr[0],astol.curr[1],astol.curr[2]])
temp=pos
pos=np.array(pos)
global_path = np.array(astol.path)
pba = np.array(pba)
phi_dot=np.array(phi_dot)

simp is proceeding to next node [20. -0. -0.]
correcting heading
heading corrected
[[  0.          -0.           0.        ]
 [ 20.          -0.          -0.        ]
 [ 40.         -20.          -0.78539816]
 [ 60.         -30.          -0.78539816]
 [ 60.         -30.          -0.78539816]]
proceeding to next node
current goal is [20. -0. -0.]
[0.56249449 0.00248891 0.00442475]
[1.12499421e+00 3.05622081e-03 1.00855064e-03]
[1.68749420e+00 3.12860020e-03 1.28674474e-04]
[ 2.24999420e+00  3.07341856e-03 -9.81006991e-05]
[ 2.81249419e+00  2.98535791e-03 -1.56552266e-04]
[ 3.37499418e+00  2.88884453e-03 -1.71579353e-04]
[ 3.93749418e+00  2.79018728e-03 -1.75390667e-04]
[ 4.49999417e+00  2.69101848e-03 -1.76300087e-04]
[ 5.06249416e+00  2.59176214e-03 -1.76455718e-04]
[ 5.62499415e+00  2.49252981e-03 -1.76413032e-04]
[ 6.18749414e+00  2.39335225e-03 -1.76315653e-04]
[ 6.74999413e+00  2.29423944e-03 -1.76200564e-04]
[ 7.31249412e+00  2.19519609e-03 -1.76077068e-04]
[ 7.87499412e+00  2.096

In [ ]:
print(simp.global_path)

In [ ]:
# pos[0,:]=np.array([0,0,0])
curr=np.array((trajectory[0][0],trajectory[0][1],np.deg2rad(angle)))
print(np.array((trajectory[0][0],trajectory[0][1],np.deg2rad(angle))))
print(curr)
# print(phi_dot[1])
print()

In [ ]:
pos=np.array(pos)
global_path = np.array(astol.path)
pba = np.array(pba)
phi_dot=np.array(phi_dot)
print(global_path)

In [ ]:
plt.subplots()
plt.plot(pos[:,0],pos[:,1],'b')
plt.plot(global_path[0,0],-global_path[0,1],'rv')
plt.plot(global_path[:,0],-global_path[:,1],'r*')
plt.plot(global_path[:,0],-global_path[:,1],'r--')
plt.plot(global_path[-1,0],-global_path[-1,1],'ro')


In [ ]:
plt.subplots()
plt.plot(pba[:,0],'g',label='rho')
plt.subplots()
plt.plot(pba[:,1],'b',label='alpha')
plt.subplots()
plt.plot(pba[:,2],'r',label='beta')


In [ ]:
plt.subplots()
plt.plot(phi_dot[:,0],'g')
plt.plot(phi_dot[:,1],'--b')
plt.show()

In [ ]:
print(phi_dot[:,])